# Data Engineering EDA Project

By Aommy, Austin, Ling, Tath (yippee)

-----------------------

## Installing Dependencies

### Minio Object Store dependencies

1. Install minio cli if not already installed.
2. Create client
3. Make bucket (if needed)

In [10]:
! pip install minio --quiet

In [1]:
from minio import Minio

# Create client with access and secret key.
# client = Minio("s3.amazonaws.com", "ACCESS-KEY", "SECRET-KEY")

# Create client with access key and secret key with specific region.
client = Minio(
    "localhost:9000",
    access_key="ROOTNAME",
    secret_key="CHANGEME123",
    secure=False
)

if client.bucket_exists("eda"):
    print("Bucket exists.")
else:
    client.make_bucket("eda")
    print("Bucket created.")

Bucket exists.


## Importing libraries

In [2]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

import io
from urllib.request import urlopen
import zipfile
import os

In [7]:
!pip install --quiet yfinance --upgrade --no-cache-dir

---------------------

## Data Collection

### Data from Global Events

Installing data from http://data.gdeltproject.org/events/index.html for global event data!

In [5]:
for y in range(19, 25):
    for m in range(1, 13):
        if m < 10:
            m = "0" + str(m)
        for d in range(1, 32):
            if d < 10:
                d = "0" + str(d)
            try:
                response = client.get_object("eda", "gdelt-parquet/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d))
                # Read data from response.
            except:
                print("File doesn't exist!")
                try:
                    data = urlopen(
                        "http://data.gdeltproject.org/events/20{0}{1}{2}.export.CSV.zip".format(y,m,d),
                    )
                    print("Uploading file!")
                    result = client.put_object(
                        "eda", "gdelt-parquet/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d), data, length=-1, part_size=10*1024*1024,
                    )
                    print(
                        "created {0} object; etag: {1}, version-id: {2}".format(
                            result.object_name, result.etag, result.version_id,
                        ),
                    )
                except:
                    print("No file to download!")
            finally:
                response.close()
                response.release_conn()

File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
Uploading file!
created gdelt/2020/04/09.export.CSV.zip object; etag: 5cf7553a7e25189741f5f55711b3a5f2, version-id: None
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download!
File doesn't exist!
No file to download

In [47]:
gdelt_headers = {
    "GlobalEventID": int,
    "Day": int,
    "MonthYear": int,
    "Year": int,
    "FractionDate": float,
    "Actor1Code": str,
    "Actor1Name": str,
    "Actor1CountryCode": str,
    "Actor1KnownGroupCode": str,
    "Actor1EthnicCode": str,
    "Actor1Religion1Code": str,
    "Actor1Religion2Code": str,
    "Actor1Type1Code": str,
    "Actor1Type2Code": str,
    "Actor1Type3Code": str,
    "Actor2Code": str,
    "Actor2Name": str,
    "Actor2CountryCode": str,
    "Actor2KnownGroupCode": str,
    "Actor2EthnicCode": str,
    "Actor2Religion1Code": str,
    "Actor2Religion2Code": str,
    "Actor2Type1Code": str,
    "Actor2Type2Code": str,
    "Actor2Type3Code": str,
    "IsRootEvent": bool,
    "EventCode": str,
    "EventBaseCode": str,
    "EventRootCode": str,
    "QuadClass": float,
    "GoldsteinScale": float,
    "NumMentions": float,
    "NumSources": float,
    "NumArticles": float,
    "AvgTone": float,
    "Actor1Geo_Type": float,
    "Actor1Geo_Fullname": str,
    "Actor1Geo_CountryCode": str,
    "Actor1Geo_ADM1Code": str,
    "Actor1Geo_Lat": float,
    "Actor1Geo_Long": object,
    "Actor1Geo_FeatureID": object,
    "Actor2Geo_Type": float,
    "Actor2Geo_Fullname": str,
    "Actor2Geo_CountryCode": str,
    "Actor2Geo_ADM1Code": str,
    "Actor2Geo_Lat": float,
    "Actor2Geo_Long": object,
    "Actor2Geo_FeatureID": object,
    "ActionGeo_Type": float,
    "ActionGeo_Fullname": str,
    "ActionGeo_CountryCode": str,
    "ActionGeo_ADM1Code": str,
    "ActionGeo_Lat": float,
    "ActionGeo_Long": object,
    "ActionGeo_FeatureID": object,
    "DateAdded": float,
    "SOURCEURL": str
}

In [30]:
len(gdelt_headers)

58

In [31]:
gdelt_headers.keys()

dict_keys(['GlobalEventID', 'Day', 'MonthYear', 'Year', 'FractionDate', 'Actor1Code', 'Actor1Name', 'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode', 'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code', 'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Code', 'Actor2Name', 'Actor2CountryCode', 'Actor2KnownGroupCode', 'Actor2EthnicCode', 'Actor2Religion1Code', 'Actor2Religion2Code', 'Actor2Type1Code', 'Actor2Type2Code', 'Actor2Type3Code', 'IsRootEvent', 'EventCode', 'EventBaseCode', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'NumMentions', 'NumSources', 'NumArticles', 'AvgTone', 'Actor1Geo_Type', 'Actor1Geo_Fullname', 'Actor1Geo_CountryCode', 'Actor1Geo_ADM1Code', 'Actor1Geo_Lat', 'Actor1Geo_Long', 'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_Fullname', 'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat', 'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type', 'ActionGeo_Fullname', 'ActionGeo_CountryCode', 'ActionGeo_ADM1Code', 'ActionGeo_

### Data from YahooFinance

In [30]:
import yfinance as yf

companies = {
    "tech": ['AAPL', 'MSFT', 'NVDA'],
    "energy": ['XOM', 'CVX', 'BP', 'COP'],
    "food": ['KO', 'PEP', 'PG'],
    "health": ['JNJ', 'PFE', 'ABBV'],
    "finance": ['JPM', 'V', 'MA'],
    "agriculture": ['DE', 'CTVA', 'ADM'],
    "telecom": ['T', 'VZ', 'TMUS'],
    "materials": ['DOW', 'NEM', 'FCX']
}

In [37]:
uploaded = False
for sector in companies:
    for stock in companies[sector]:
        try:
            response = client.get_object("eda", "yfinance/{0}/{1}.snappy.parquet".format(sector,stock))
            # Read data from response.
        except:
            print("File doesn't exist!")
            data = yf.download(stock, start="2019-01-01", end="2024-10-15", progress=False)
            result = client.put_object(
                "eda", "yfinance/{0}/{1}.snappy.parquet".format(sector,stock), io.BytesIO(data.to_parquet(compression='snappy')), length=-1, part_size=10*1024*1024,
            )
            print(
                "created {0} object; etag: {1}, version-id: {2}".format(
                    result.object_name, result.etag, result.version_id,
                ),
            )
            uploaded = True
        finally:
            response.close()
            response.release_conn()
if (not uploaded):
    print("No changes to make.")

No changes to make.


----------------------

## Data Processing

In [50]:
# resp = client.get_object("eda", "gdelt/2019/01/01.export.CSV.zip")
# from io import BytesIO

# df = pd.read_csv(BytesIO(resp.data))

In [52]:
for y in range(19, 25):
    for m in range(1, 13):
        if m < 10:
            m = "0" + str(m)
        for d in range(1, 32):
            if d < 10:
                d = "0" + str(d)
            try:
                response = client.get_object("eda", "gdelt-parquet/20{0}/{1}/{2}.snappy.parquet".format(y,m,d))
                # Read data from response.
            except:
                print("File doesn't exist! 20{0}/{1}/{2}".format(y,m,d))
                try:
                    response = client.get_object("eda", "gdelt/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d))
                    client.fget_object("eda", "gdelt/20{0}/{1}/{2}.export.CSV.zip".format(y,m,d), 
                                       "./data/gdelt/20{0}{1}{2}.export.CSV.zip".format(y,m,d))
                    with zipfile.ZipFile("./data/gdelt/20{0}{1}{2}.export.CSV.zip".format(y,m,d),"r") as zip_ref:
                        zip_ref.extractall("./data/gdelt/temp")
                    data = pd.read_csv("./data/gdelt/temp/20{0}{1}{2}.export.CSV".format(y,m,d), sep='\t', header=None, names=gdelt_headers.keys(), dtype=gdelt_headers, on_bad_lines='warn')
                    result = client.put_object(
                        "eda", "gdelt-parquet/20{0}/{1}/{2}.snappy.parquet".format(y,m,d), io.BytesIO(data.to_parquet(compression='snappy')), length=-1, part_size=10*1024*1024,
                    )
                    print(
                        "created {0} object; etag: {1}, version-id: {2}".format(
                            result.object_name, result.etag, result.version_id,
                        ),
                    )
                    os.remove("./data/gdelt/temp/20{0}{1}{2}.export.CSV".format(y,m,d))
                    os.remove("./data/gdelt/20{0}{1}{2}.export.CSV.zip".format(y,m,d))
                except:
                    print("This date does not exist!!!")

File doesn't exist! 2019/02/29
This date does not exist!!!
File doesn't exist! 2019/02/30
This date does not exist!!!
File doesn't exist! 2019/02/31
This date does not exist!!!
File doesn't exist! 2019/03/01
created gdelt-parquet/2019/03/01.snappy.parquet object; etag: 3128ad7b4d2cb9e8c5fed7f7d150515e-2, version-id: None
File doesn't exist! 2019/03/02
created gdelt-parquet/2019/03/02.snappy.parquet object; etag: 63d1032bd0a45ba09a86ffc00324484b, version-id: None
File doesn't exist! 2019/03/03
created gdelt-parquet/2019/03/03.snappy.parquet object; etag: aa3a478f9f4e209f497b746169cd7e0f, version-id: None
File doesn't exist! 2019/03/04
created gdelt-parquet/2019/03/04.snappy.parquet object; etag: 3b549267a3cdec0992c10c5dc540409c-2, version-id: None
File doesn't exist! 2019/03/05
created gdelt-parquet/2019/03/05.snappy.parquet object; etag: 850e25213e64d7091ee7c88f68640c2d-2, version-id: None
File doesn't exist! 2019/03/06
created gdelt-parquet/2019/03/06.snappy.parquet object; etag: 82da

In [49]:
data

,GlobalEventID,Day,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_Fullname,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DateAdded,SOURCEURL
0,824358302,20180217,201802,2018,2018.1288,AUSAGR,AUSTRALIA,AUS,NaN,NaN,...,-1555351,4.0,"Melbourne, Victoria, Australia",AS,AS07,-37.816700,144.967,-1586844,20190217.0,https://www.canberratimes.com.au/national/vict...
1,824358303,20180217,201802,2018,2018.1288,AUSAGR,MELBOURNE,AUS,NaN,NaN,...,-1586844,4.0,"Melbourne, Victoria, Australia",AS,AS07,-37.816700,144.967,-1586844,20190217.0,https://www.canberratimes.com.au/national/vict...
2,824358304,20180217,201802,2018,2018.1288,CHN,CHINA,CHN,NaN,NaN,...,-2960561,4.0,"Tehran, Tehran, Iran",IR,IR26,35.750000,51.5148,10074674,20190217.0,http://www.reflector.com/National-Business/201...
3,824358305,20180217,201802,2018,2018.1288,CHN,CHINA,CHN,NaN,NaN,...,10074674,4.0,"Tehran, Tehran, Iran",IR,IR26,35.750000,51.5148,10074674,20190217.0,https://www.americanpress.com/wire/the-latest-...
4,824358306,20180217,201802,2018,2018.1288,CHN,CHINA,CHN,NaN,NaN,...,10074674,4.0,"Tehran, Tehran, Iran",IR,IR26,35.750000,51.5148,10074674,20190217.0,http://www.reflector.com/National-Business/201...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103350,824518312,20190217,201902,2019,2019.1288,asy,ASSYRIAN,NaN,NaN,asy,...,NaN,1.0,United States,US,US,39.828175,-98.5795,US,20190217.0,https://medium.com/@christina.nyoseph/on-the-c...
103351,824518313,20190217,201902,2019,2019.1288,chm,MARI,NaN,NaN,chm,...,NaN,4.0,"Kambuzuma, Harare, Zimbabwe",ZI,ZI10,-17.854400,30.9672,124324,20190217.0,https://www.herald.co.zw/zim-writers-mourn-fal...
103352,824518314,20190217,201902,2019,2019.1288,chm,MARI,NaN,NaN,chm,...,-2007087,4.0,"Farai, Nigeria (general), Nigeria",NI,NI00,9.450000,12.1167,-2007087,20190217.0,https://www.herald.co.zw/zim-writers-mourn-fal...
103353,824518315,20190217,201902,2019,2019.1288,chr,CHEROKEE,NaN,NaN,chr,...,531871,3.0,"Washington, District of Columbia, United States",US,USDC,38.895100,-77.0364,531871,20190217.0,http://wondradio.com/abc-politics/e8063973baa8...


----------------------

## Data Analysis